# Intro to sympy


SymPy is a library that makes it possible to do symbolic calculations in Python. It comes pre-installed with Anaconda, or can be installed with `pip install sympy`.

In [1]:
import sympy as sp

Fisrtly, we can use some of IPythons built in functions to get easily readable and pretty print outs. IPyton allows for LaTeX printout of sympy expressions. If there is only one expression in a cell, the function below is not necessary. However, it is usefull for printing more than one expression.

In [2]:
from IPython.display import display, Latex

def print_expression(S, name=""):
    """Print out a single sympy expression"""
    string = "$$" 
    if name!="": string = string + name + "=" 
    string = string + sp.latex(S) + "$$"
    return display(Latex(string))

The basic type in sympy is the symbols. Out of these we can make expressions, and manipulate them.

In [3]:
a, b, c, x, y, z = sp.symbols("a, b, c, x, y, z")
f = 2 + x**2*y + sp.exp(x + y)
f

x**2*y + exp(x + y) + 2

In [4]:
f.diff(x).diff(y)

2*x + exp(x + y)

In [5]:
sp.integrate(f, (x, 0, a))

a**3*y/3 + 2*a - exp(y) + exp(a + y)

In [6]:
g = sp.lambdify([x, y], f) # make a regular python funciton
print(g(1, 1))

10.38905609893065


We can manipulate arbitrary order symbolic arrays with the Array class 

In [7]:
A = sp.Array([[x, x*y], [sp.sin(x), x**y]])
print_expression(A)

# Differentiate all elements of the array
print_expression(A.diff(x))

# Appøy a function on each element of the array
print_expression(A.applyfunc(lambda x : x**2))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>


Matrix product, or a more general contraction of indecies, is done by first a tenosr product, then a contraction. For example, the matrix product $(AB)_{ik} = \sum_j A_{ij} B_{jk}$ can be done by definining an new array,
$$
    M_{ijk\ell} = \left( A \otimes B \right)_{ijk\ell} = A_{i j} B_{k \ell},
$$
Then contracting the middle indecies,
$$
    (AB)_{ik} = \sum_j M_{ijjk}
$$

In [8]:
from sympy import tensorproduct as tp
from sympy import tensorcontraction as tc

B = sp.Array([[x**2, y - x], [y, sp.exp(y/x)]])

print_expression(tc(tp(A, B), (1, 2)))

<IPython.core.display.Latex object>

If the array is 2D ($n\times m$) there is more funcitonality like eigenvectors and eigenvalues, in the `Matrix` calss. This array is not quite as rich as the numpy array. You can numpy arrays with sympy symbols, if anything is missing.

Indexed objects gives the possibility manipulate tensors with symbolic indecies, wich ranges from $0$ to $n - 1$

In [9]:
# Horrible hack to print indexed objects, courtesy StackOverflow: 
# https://stackoverflow.com/questions/61470842/strange-printing-in-sympy-for-indexed-variables
# This breaks the print_expression function :(

from sympy import *
from sympy.printing.latex import LatexPrinter

class CustomLatexPrinter(LatexPrinter):
    def _print_Idx(self, expr):
        return expr.name

    @classmethod
    def printer(cls, expr, **kwargs):
        return cls(kwargs).doprint(expr)

init_printing(use_latex='mathjax', latex_printer=CustomLatexPrinter.printer)

In [10]:
i, j, m, n = sp.symbols("i, j, m, n", integer=True)
i = sp.Idx(i, n)    # A symbolic index
j = sp.Idx(j, n)
z = sp.IndexedBase("z")    # An object with indecies, like a tensor
z[i].diff(z[j])

δ   
 i,j

In [11]:
f = sp.sqrt(sp.Sum(z[i]**2, (i, 0, n-1)))
f

         _____________
        ╱ n - 1       
       ╱   ___        
      ╱    ╲          
     ╱      ╲       2 
    ╱       ╱   z[i]  
   ╱       ╱          
  ╱        ‾‾‾        
╲╱        i = 0       

In [12]:
f.diff(z[j])

   n - 1                
    ___                 
    ╲                   
     ╲   2⋅δ   ⋅z[i]    
     ╱      i,j         
    ╱                   
    ‾‾‾                 
   i = 0                
────────────────────────
           _____________
          ╱ n - 1       
         ╱   ___        
        ╱    ╲          
       ╱      ╲       2 
2⋅    ╱       ╱   z[i]  
     ╱       ╱          
    ╱        ‾‾‾        
  ╲╱        i = 0       